<a href="https://colab.research.google.com/github/Kaleab1999/Medical-Transcription-for-medicne-department/blob/main/Bio_bert_for_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [2]:
import re
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [4]:
df= pd.read_csv("/content/preproceed MT file.csv")
df.head()

,Unnamed: 0,transcription,medical_specialty,tokenised,stemmed,stemmed_without_stop
0,3,d m mode left atrial enlargement with left a...,Cardiovascular / Pulmonary,"['d', 'm', 'mode', 'left', 'atrial', 'enlargem...","['d', 'm', 'mode', 'left', 'atrial', 'enlarg',...",mode left atrial enlarg left atrial diamet cm ...
1,4,the left ventricular cavity size and wall thi...,Cardiovascular / Pulmonary,"['the', 'left', 'ventricular', 'cavity', 'size...","['the', 'left', 'ventricular', 'caviti', 'size...",left ventricular caviti size wall thick appear...
2,7,d echocardiogram multiple views of the heart ...,Cardiovascular / Pulmonary,"['d', 'echocardiogram', 'multiple', 'views', '...","['d', 'echocardiogram', 'multipl', 'view', 'of...",echocardiogram multipl view heart great vessel...
3,9,description normal cardiac chambers size nor...,Cardiovascular / Pulmonary,"['description', 'normal', 'cardiac', 'chambers...","['descript', 'normal', 'cardiac', 'chamber', '...",descript normal cardiac chamber size normal le...
4,11,d study mild aortic stenosis widely calcifie...,Cardiovascular / Pulmonary,"['d', 'study', 'mild', 'aortic', 'stenosis', '...","['d', 'studi', 'mild', 'aortic', 'stenosi', 'w...",studi mild aortic stenosi wide calcifi minim r...


In [7]:
model_name = 'dmis-lab/biobert-base-cased-v1.1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

In [8]:
df['medical_specialty'].value_counts()

 Cardiovascular / Pulmonary    371
 General Medicine              259
 Gastroenterology              224
 Neurology                     223
Name: medical_specialty, dtype: int64

In [9]:
df.shape

(1077, 6)

In [11]:
data= df[['stemmed_without_stop','medical_specialty']]
data.head()

,stemmed_without_stop,medical_specialty
0,mode left atrial enlarg left atrial diamet cm ...,Cardiovascular / Pulmonary
1,left ventricular caviti size wall thick appear...,Cardiovascular / Pulmonary
2,echocardiogram multipl view heart great vessel...,Cardiovascular / Pulmonary
3,descript normal cardiac chamber size normal le...,Cardiovascular / Pulmonary
4,studi mild aortic stenosi wide calcifi minim r...,Cardiovascular / Pulmonary


In [12]:
data.isnull().sum()

stemmed_without_stop    0
medical_specialty       0
dtype: int64

In [14]:
data.iloc[[6]]

,stemmed_without_stop,medical_specialty
6,histori present ill patient year old femal pre...,General Medicine


In [15]:
X= data['stemmed_without_stop'].tolist()
y= list( data['medical_specialty'].astype('category').cat.codes)

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2)

### model building

In [27]:
train_d= tokenizer(X_train, padding= True, truncation= True, max_length=512)
test_d= tokenizer(X_test, padding= True, truncation= True, max_length=512)

In [28]:
train_d.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

#### create dataset

In [29]:
import torch

In [30]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [31]:
train_dataset = Dataset(train_d, y_train)
test_dataset = Dataset(test_d, y_test)

In [32]:
train_dataset[5]

{'input_ids': tensor([  101,  2581,   185,  7889,  5053,  1665, 12211,  1394,  2552,  2552,
         17299,  2999,  2942,  5048, 14494, 14255, 20327, 12416, 14044, 25669,
         15391,  1161,  2880,   171,  5412,  1182, 15170,  1197,   188, 10536,
          1611,  2999,  1653,  2942,   179,  3984, 12090,  1665, 11184,  4490,
          2330,  1443,  8241,  1988, 11602,  4463,  1231, 20080,  4199,  1609,
           178,  2047,  2999,  2942,  8241,  1988, 16557,  5383,  2330,  2495,
          1665, 10205, 26862,  2999,  5841,  1182,  2330,  3811,  4252, 16748,
          3811, 14044, 25669, 15391,  1161,  5048, 14494,  8241,  1988,  3811,
          7684,  8362, 12809,  2050,  5082,  5822,  1443,  5048, 14494,  6187,
          7111,  1403,  8241,  1988,   189, 17162, 10224,  1143, 12913,  4047,
          2330,  2999,  1609, 27820,  2286, 28054,  3811,   174,  3101,  1361,
          3678,  4252, 16748,  3678,   188, 17162, 11006,  1197,  2241,  8241,
          1988,  9468,  1874,  1501,  1

In [33]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    # average= 'micro' because it's multi-class classification u can use also macro, or weight or other 
    # u shouldn't mention anything for binary classification
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

    return { "precision": precision, "recall": recall, 'f1-score': f1, "acc": accuracy}

In [34]:
# Define Trainer
args = TrainingArguments(
    output_dir="reslt_",
    num_train_epochs=3,
    per_device_train_batch_size=16

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [35]:
trainer.train()

***** Running training *****
  Num examples = 861
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 162
  Number of trainable parameters = 108313348


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=162, training_loss=0.4690146740571952, metrics={'train_runtime': 262.2075, 'train_samples_per_second': 9.851, 'train_steps_per_second': 0.618, 'total_flos': 679628059987968.0, 'train_loss': 0.4690146740571952, 'epoch': 3.0})

In [36]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 216
  Batch size = 8


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.3587670624256134,
 'eval_precision': 0.8703703703703703,
 'eval_recall': 0.8703703703703703,
 'eval_f1-score': 0.8703703703703703,
 'eval_acc': 0.8703703703703703,
 'eval_runtime': 7.4541,
 'eval_samples_per_second': 28.977,
 'eval_steps_per_second': 3.622,
 'epoch': 3.0}

In [37]:
trainer.save_model('BioBERT for MT')

Saving model checkpoint to BioBERT for MT
Configuration saved in BioBERT for MT/config.json
Model weights saved in BioBERT for MT/pytorch_model.bin


In [ ]:
!pip install huggingface_hub

In [39]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [43]:
model_2.push_to_hub("Kaleab1999/BioBERT-For_MT")

Configuration saved in /tmp/tmpd3mogn1k/config.json
Model weights saved in /tmp/tmpd3mogn1k/pytorch_model.bin
Uploading the following files to Kaleab1999/BioBERT-For_MT: pytorch_model.bin,config.json


pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/Kaleab1999/BioBERT-For_MT/commit/ef0f7013341a75c65d018f01518476eaf3870336', commit_message='Upload BertForSequenceClassification', commit_description='', oid='ef0f7013341a75c65d018f01518476eaf3870336', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_2 = AutoModelForSequenceClassification.from_pretrained("/content/BioBERT for MT")